## 1. Preliminary

### 1.1 Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance

# Options pandas
pd.options.mode.chained_assignment = None

### 1.2 Data

In [2]:
def read_csv(filename, delimiter=";"):
    """
    Read a CSV file and return a DataFrame.
    
    Parameters:
    - filename (str): Path to the CSV file.
    - delimiter (str): Delimiter used in the CSV file (default: ",").
    
    Returns:
        pd.DataFrame or None: DataFrame containing the data read from the CSV file,
                              or None if the file is not found.
    """
    try:
        return pd.read_csv(filename, delimiter=delimiter)
    except FileNotFoundError:
        print(f"File '{filename}' not found.")
    return None

In [3]:
def clean_data(df):
    """
    Clean the DataFrame by dropping NaN values and handling duplicates.
    
    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    
    Returns:
        pd.DataFrame or None: Cleaned DataFrame, or None if input DataFrame is None.
    """
    if df is None:
        return None
    df = df.astype(str)
    df = df.dropna()
    if df.duplicated().any():
        print("Duplicate values found in the dataset.")
    return df

In [4]:
def calculate_distance(df, target_coords=(46.227638, 2.213749)):
    """
    Calculate the distance between each location in the DataFrame and a target location.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing latitude and longitude columns.
    - target_coords (tuple): Coordinates (latitude, longitude) of the target location (default: France).
    
    Returns:
        pd.DataFrame or None: DataFrame with distance calculated for each location,
                              or None if input DataFrame is None.
    """
    if df is None:
        return None
    
    # Drop rows with NaN values in latitude or longitude columns
    df = df.dropna(subset=["latitude", "longitude"])
    
    # Check if any NaN values remain after dropping
    if df.isnull().values.any():
        print("NaN values found after dropping rows with missing coordinates.")
        print(df[df.isnull().any(axis=1)])
        return None
    
    distances = []
    for _, row in df.iterrows():
        try:
            distance = geopy.distance.distance((row.latitude, row.longitude), target_coords).km
            distances.append(distance)
        except ValueError:
            distances.append(np.nan)
    
    df["km_from_target"] = distances
    return df

In [5]:
def clean_column_names(df, column_mapping):
    """
    Clean column names of the DataFrame using the provided mapping.
    
    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_mapping (dict): Mapping of old column names to new column names.
    
    Returns:
        pd.DataFrame or None: DataFrame with cleaned column names,
                              or None if input DataFrame is None.
    """
    if df is None:
        return None
    df.rename(columns=column_mapping, inplace=True)
    return df

In [6]:
def correct_country_names(datasets, country_name, correct_name):
    """
    Correct inconsistent country names in multiple datasets.
    
    Args:
    - datasets (list): List of DataFrames to correct country names.
    - country_name (str): Incorrect country name.
    - correct_name (str): Correct country name.
    
    Returns:
        list: List of corrected DataFrames.
    """
    if not isinstance(datasets, list):
        raise ValueError("Input 'datasets' must be a list of DataFrames.")
    
    corrected_datasets = []
    for df in datasets:
        if 'Zone' in df.columns:
            df['Zone'] = df['Zone'].replace({country_name: correct_name})
        corrected_datasets.append(df)
    return corrected_datasets

In [7]:
def reorganize(df, mappings=None, output_columns=None):
    """
    Reorganize DataFrame with specified mappings and output columns.
    
    Args:
    - df (pd.DataFrame): Input DataFrame.
    - mappings (dict): Mapping of old column names to new column names.
    - output_columns (list): List of column names to keep in the output DataFrame.
    
    Returns:
        pd.DataFrame: Reorganized DataFrame.
    """
    if mappings:
        df = df.rename(columns=mappings)
    if output_columns:
        df = df[output_columns]
    return df

In [8]:
def save_csv(df, filename):
    """
    Save the DataFrame to a CSV file.
    
    Parameters:
    - df (pd.DataFrame): DataFrame to be saved.
    - filename (str): Path to save the CSV file.
    """
    if df is not None:
        df.to_csv(filename, index=False)

In [9]:
political_stability = read_csv("political_stability.csv")
exchange_rate = read_csv("exchange_rates.csv")
consumer_price_index = read_csv("consumer_price_indices.csv")
population = read_csv("Population_2000_2018.csv")
food_availability = read_csv("DisponibiliteAlimentaire_2017.csv")
country_coordinates = read_csv("lat_long_pays.csv")
emissions_totals = read_csv("emissions_totals.csv")

In [10]:
political_stability, exchange_rate, consumer_price_index, population, food_availability, country_coordinates, emissions_totals = correct_country_names(
    [political_stability, exchange_rate, consumer_price_index, population, food_availability, country_coordinates, emissions_totals],
    'Sudan (former)',
    'Sudan'
)
political_stability, exchange_rate, consumer_price_index, population, food_availability, country_coordinates, emissions_totals = correct_country_names(
    [political_stability, exchange_rate, consumer_price_index, population, food_availability, country_coordinates, emissions_totals],
    'Sudan (ex)',
    'Sudan'
)

In [11]:
political_stability = clean_data(political_stability)
exchange_rate = clean_data(exchange_rate)
consumer_price_index = clean_data(consumer_price_index)
population = clean_data(population)
food_availability = clean_data(food_availability)
country_coordinates = clean_data(country_coordinates)
country_coordinates = calculate_distance(country_coordinates)
emissions_totals = clean_data(emissions_totals)

In [12]:
column_mappings = {
    "name": "Zone",
    "latitude": "Latitude",
    "longitude": "Longitude",
    "km_from_target": "Km - Par rapport à la France"
    # Add other mappings as needed
}

In [13]:
population = reorganize(population, output_columns=["Zone", "Année", "Valeur"])
food_availability = reorganize(food_availability, output_columns=["Zone", "Année", "Élément", "Produit", "Valeur"])
country_coordinates = reorganize(country_coordinates, mappings=column_mappings)
exchange_rate = reorganize(exchange_rate, mappings={"Area": "Zone", "Currency": "Devise", "Element": "Unité", "Year": "Année", "Value": "Valeur"}, output_columns=["Zone", "Devise", "Année", "Unité", "Valeur"])
consumer_price_index = reorganize(consumer_price_index, mappings={"Area": "Zone", "Year": "Année", "Months": "Mois", "Item": "Objet", "Value": "Valeur"}, output_columns=["Zone", "Année", "Mois", "Objet", "Valeur"])
political_stability = reorganize(political_stability, mappings={"Country": "Zone", "Year": "Année", "Political_Stability": "Stabilité Politique", "Granularity": "Granularité"}, output_columns=["Zone", "Année", "Stabilité Politique"])
emissions_totals = reorganize(emissions_totals, mappings={"Area": "Zone", "Element": "Elément", "Year": "Année", "Item": "Objet", "Value": "Valeur"}, output_columns=["Zone", "Elément", "Objet", "Année", "Valeur"])

## 3. Data exploration

### 3.1 Population

In [14]:
def explore_population_year(population, year="2017"):
    """
    Explore population for a specific year and return the DataFrame with top countries.
    
    Args:
    - data (DataFrame): DataFrame containing population data.
    - year (str): Year for which population data is to be explored.
    
    Returns:
        DataFrame: Population data for the specified year.
    """
    population_year = population[population["Année"] == year]
    population_by_country = population_year.groupby("Zone").sum()["Valeur"].reset_index()
    population_by_country.rename(columns={"Valeur": "Population"}, inplace=True)
    
    # Print results
    print(f"Population for the year {year}:")
    print(population_by_country.head().to_string(index=False), "\n[..]")
    
    return population_by_country

In [15]:
population_result = explore_population_year(population)

Population for the year 2017:
          Zone Population
   Afghanistan  36296.113
       Albania   2884.169
       Algeria  41389.189
American Samoa      55.62
       Andorra     77.001 
[..]


In [16]:
def explore_population_growth(population):
    """
    Explore population growth and print top and bottom countries.

    Args:
    - data (DataFrame): DataFrame containing population data.
    """
    # Pivot table to analyze population growth over years
    pop_evol = population.pivot_table(index="Zone", columns="Année", values="Valeur")

    # Calculate percentage change in population for each country
    pop_evol_pct_change = pop_evol.pct_change(axis='columns')

    # Calculate average population growth over the years for each country
    pop_evol["Change Average"] = pop_evol_pct_change.mean(axis=1)

    # Sort the DataFrame by the 'Change Average' column
    pop_evol_sorted = pop_evol["Change Average"].sort_values(ascending=False)

    # Extract the top and bottom countries
    top_growth_countries = pop_evol_sorted.head(3)
    bottom_growth_countries = pop_evol_sorted.tail(3)

    # Print results
    print("Countries with the highest population growth:")
    for country, growth in top_growth_countries.iteritems():
        print(f"{country}: {growth:.2f}")

    print("\nCountries with the lowest population growth:")
    for country, growth in bottom_growth_countries.iteritems():
        print(f"{country}: {growth:.2f}")
        
    return pop_evol_sorted

In [17]:
population_growth_result = explore_population_growth(population)

Countries with the highest population growth:
Qatar: 0.09
United Arab Emirates: 0.07
Bahrain: 0.05

Countries with the lowest population growth:
Latvia: -0.01
Lithuania: -0.01
Wallis and Futuna: -0.01


### 3.2 Food Availability

In [18]:
def explore_poultry_availability(food_availability):
    """
    Explore poultry availability and print top and bottom countries.
    
    Args:
    - data (DataFrame): DataFrame containing food availability data.
    """
    
    # Filter production data for the year 2017
    poultry_availability_2017 = food_availability[(food_availability["Année"] == "2017") & 
                                                   (food_availability["Élément"] == "Disponibilité alimentaire (Kcal/personne/jour)") & 
                                                   (food_availability["Produit"] == "Viande de Volailles")]
    
    # Group poultry availability by country and sum the values
    poultry_per_country = poultry_availability_2017.groupby("Zone").sum()["Valeur"].reset_index() 
    
    # Convert "Valeur" column to numeric type
    poultry_per_country["Valeur"] = pd.to_numeric(poultry_per_country["Valeur"])
    
    # Sort the DataFrame by the 'Valeur' column
    poultry_per_country_sorted = poultry_per_country.sort_values(by="Valeur", ascending=False).reset_index(drop=True)

    # Extract the top and bottom countries
    top_poultry_countries = poultry_per_country_sorted.head(3)
    bottom_poultry_countries = poultry_per_country_sorted.tail(3)

    # Print results
    print("Countries with the highest poultry availability:")
    for country, value in zip(top_poultry_countries["Zone"], top_poultry_countries["Valeur"]):
        print(f"{country}: {value}")

    print("\nCountries with the lowest or no poultry availability:")
    for country, value in zip(bottom_poultry_countries["Zone"], bottom_poultry_countries["Valeur"]):
        print(f"{country}: {value}")

    return poultry_per_country_sorted

In [19]:
poultry_availability_result = explore_poultry_availability(food_availability)

Countries with the highest poultry availability:
Saint Vincent and the Grenadines: 243.0
Saint Lucia: 241.0
Israel: 234.0

Countries with the lowest or no poultry availability:
Nigeria: 2.0
Kenya: 2.0
Ethiopia: 0.0


In [20]:
def explore_poultry_availability(food_availability, population_result):
    """
    Explore poultry availability per capita and print top and bottom countries.
    
    Args:
    - food_availability (DataFrame): DataFrame containing food availability data.
    - population_by_country (DataFrame): DataFrame containing population data for each country.
    """
    # Filter poultry availability data
    poultry_availability_2017 = food_availability[(food_availability["Année"] == "2017") & 
                                                   (food_availability["Élément"] == "Disponibilité alimentaire (Kcal/personne/jour)") & 
                                                   (food_availability["Produit"] == "Viande de Volailles")]
    
    # Group poultry availability by country and sum the values
    poultry_per_country = poultry_availability_2017.groupby("Zone").sum()["Valeur"].reset_index()

    # Merge poultry availability and population data
    merged_data = pd.merge(poultry_availability_result, population_result, on="Zone", how="right")
    
    # Rename columns
    merged_data.rename(columns={"Valeur": "Poultry Availability"}, inplace=True)
    
    # Convert columns to numeric
    merged_data["Poultry Availability"] = pd.to_numeric(merged_data["Poultry Availability"], errors="coerce")
    merged_data["Population"] = pd.to_numeric(merged_data["Population"], errors="coerce")
    
    # Drop rows where poultry availability is NaN
    merged_data.dropna(subset=["Poultry Availability"], inplace=True)

    # Calculate availability per capita
    merged_data["Poultry_Availability_per_Capita"] = merged_data["Poultry Availability"] / merged_data["Population"]
    
    # Sort by availability per capita
    sorted_data = merged_data.sort_values(by="Poultry_Availability_per_Capita", ascending=False).reset_index(drop=True)
    
    # Extract the top and bottom countries
    top_poultry_countries = sorted_data.head(3)
    bottom_poultry_countries = sorted_data[sorted_data["Poultry_Availability_per_Capita"] > 0.00].tail(3)

    zero_poultry_countries = sorted_data[sorted_data["Poultry_Availability_per_Capita"] == 0.00]

    # Print results
    print("Countries with highest poultry availability per capita:")
    for index, row in top_poultry_countries.iterrows():
        print(f"{row['Zone']}: {row['Poultry_Availability_per_Capita']:.2f}")

    print("\nCountries with lowest poultry availability per capita:")
    for index, row in bottom_poultry_countries.iterrows():
        print(f"{row['Zone']}: {row['Poultry_Availability_per_Capita']:.2f}")
    
    print("\nCountries with no poultry availability per capita:")
    for index, row in zero_poultry_countries.iterrows():
        print(f"{row['Zone']}: {row['Poultry_Availability_per_Capita']:.2f}")
        
    return sorted_data

In [21]:
poultry_availability_capita_result = explore_poultry_availability(food_availability, population_result)

Countries with highest poultry availability per capita:
Saint Kitts and Nevis: 3.65
Antigua and Barbuda: 2.44
Saint Vincent and the Grenadines: 2.21

Countries with lowest poultry availability per capita:
Kenya: 0.00
Nigeria: 0.00
India: 0.00

Countries with no poultry availability per capita:
Ethiopia: 0.00


In [22]:
def poultry_import_by_country(food_availability):
    """
    Calculate poultry imports by country.
    
    Args:
    - data (DataFrame): DataFrame containing food availability data.
        
    Returns:
        DataFrame: Poultry imports by country.
    """
    # Filter imports data
    poultry_imports = food_availability[(food_availability["Année"] == "2017") & (food_availability["Élément"] == "Importations - Quantité") & (food_availability["Produit"] == "Viande de Volailles")]
    
    # Group imports by country and sum the values
    imports_by_country = poultry_imports.groupby("Zone").sum()["Valeur"].reset_index()
    
    # Convert "Valeur" column to numeric
    imports_by_country["Valeur"] = pd.to_numeric(imports_by_country["Valeur"])
    
    imports_by_country = imports_by_country.sort_values("Valeur", ascending=False).reset_index(drop=True)
    
    # Extract the top and bottom countries
    top_imports_countries = imports_by_country.head(3)
    bottom_imports_countries = imports_by_country[imports_by_country["Valeur"] > 0.00].tail(3)
    
    zero_imports_countries = imports_by_country[imports_by_country["Valeur"] == 0.00]

    # Print results
    print("Countries with the highest poultry imports:")
    for index, row in top_imports_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")

    print("\nCountries with the lowest poultry imports:")
    for index, row in bottom_imports_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")
        
    print("\nCountries with no poultry imports:")
    for index, row in zero_imports_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")
            
    return imports_by_country

In [23]:
poultry_imports_result = poultry_import_by_country(food_availability)

Countries with the highest poultry imports:
Japan: 1069.00
Mexico: 972.00
China - Hong-Kong: 907.00

Countries with the lowest poultry imports:
Bolivia (Plurinational State of): 1.00
Chad: 1.00
Indonesia: 1.00

Countries with no poultry imports:
Ecuador: 0.00
Kenya: 0.00
Israel: 0.00
Madagascar: 0.00
Malawi: 0.00
Guyana: 0.00
Bangladesh: 0.00
Burkina Faso: 0.00
Uganda: 0.00
Senegal: 0.00
Nepal: 0.00
Democratic People's Republic of Korea: 0.00
Nigeria: 0.00
Belize: 0.00
India: 0.00
Sri Lanka: 0.00
Cameroon: 0.00
Rwanda: 0.00
Tunisia: 0.00


In [24]:
def poultry_production_by_country(food_availability):
    """
    Calculate poultry production by country.
    
    Args:
    - data (DataFrame): DataFrame containing food availability data.
        
    Returns:
        DataFrame: Poultry production by country.
    """
    # Filter production data
    production_poultry = food_availability[(food_availability["Année"] == "2017") & (food_availability["Élément"] == "Production") & (food_availability["Produit"] == "Viande de Volailles")]
    
    # Group production by country and sum the values
    production_by_country = production_poultry.groupby("Zone").sum()["Valeur"].reset_index()
    
    # Convert "Valeur" column to numeric
    production_by_country["Valeur"] = pd.to_numeric(production_by_country["Valeur"])
    
    production_by_country = production_by_country.sort_values("Valeur", ascending=False).reset_index(drop=True)
    
    # Extract the top and bottom countries
    top_production_countries = production_by_country.head(3)
    bottom_production_countries = production_by_country[production_by_country["Valeur"] > 0.00].tail(3).tail(3)
    
    zero_production_countries = production_by_country[production_by_country["Valeur"] == 0.00]

    # Print results
    print("Countries with the highest poultry production:")
    for index, row in top_production_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")

    print("\nCountries with the lowest production:")
    for index, row in bottom_production_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")
            
    print("\nCountries with no poultry production:")
    for index, row in zero_production_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")
        
    return production_by_country

In [25]:
poultry_production_result = poultry_production_by_country(food_availability)

Countries with the highest poultry production:
United States of America: 21914.00
China: 18236.00
Brazil: 14201.00

Countries with the lowest production:
Kiribati: 1.00
French Polynesia: 1.00
Sao Tome and Principe: 1.00

Countries with no poultry production:
Antigua and Barbuda: 0.00
Solomon Islands: 0.00
Saint Kitts and Nevis: 0.00
Mongolia: 0.00
Saint Vincent and the Grenadines: 0.00
Luxembourg: 0.00
Dominica: 0.00
Samoa: 0.00


In [26]:
def poultry_domestic_availability(food_availability):
    """
    Calculate domestic availability of poultry.
    
    Args:
    - data (DataFrame): DataFrame containing food availability data.
        
    Returns:
        DataFrame: Domestic availability of poultry by country.
    """
    # Filter domestic availability data 
    domestic_availability = food_availability[(food_availability["Année"] == "2017") & (food_availability["Élément"] == "Disponibilité intérieure") & (food_availability["Produit"] == "Viande de Volailles")]
    
    # Group domestic availability by country and sum the values
    domestic_availability_by_country = domestic_availability.groupby("Zone").sum()["Valeur"].reset_index()
    
    # Convert "Valeur" column to numeric
    domestic_availability_by_country["Valeur"] = pd.to_numeric(domestic_availability_by_country["Valeur"])
    
    domestic_availability_by_country = domestic_availability_by_country.sort_values("Valeur", ascending=False).reset_index(drop=True)
    
    # Extract the top and bottom countries
    top_domestic_countries = domestic_availability_by_country.head(3)
    bottom_domestic_countries = domestic_availability_by_country.tail(3)
    
    # Print results
    print("Countries with the highest poultry domestic availability:")
    for index, row in top_domestic_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")
        
    print("\nCountries with the lowest or no poultry domestic availability:")
    for index, row in bottom_domestic_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")
        
    return domestic_availability_by_country

In [27]:
poultry_domestic_result = poultry_domestic_availability(food_availability)

Countries with the highest poultry domestic availability:
United States of America: 18266.00
China: 18161.00
Brazil: 9982.00

Countries with the lowest or no poultry domestic availability:
Solomon Islands: 3.00
Sao Tome and Principe: 2.00
Kiribati: 2.00


### 3.3 Exchange Rate

In [28]:
def exchange_rate_by_country(exchange_rate):
    """
    Calculate exchange rates by country.
    
    Args:
    - data (DataFrame): DataFrame containing exchange rate data.
        
    Returns:
        DataFrame: Exchange rates by country.
    """
    # Filter the DataFrame for the year 2017 and local currency units per USD
    exchange_rate_2017 = exchange_rate[(exchange_rate["Année"] == "2017") & (exchange_rate["Unité"] == "Local currency units per USD")]

    # Group by country and sum the exchange rates
    exchange_rate_by_country = exchange_rate_2017.groupby(by="Zone").sum()["Valeur"].reset_index()

    # Convert "Valeur" column to numeric
    exchange_rate_by_country["Valeur"] = pd.to_numeric(exchange_rate_by_country["Valeur"])
    
    # Sort the DataFrame by exchange rate in descending order
    exchange_rate_by_country = exchange_rate_by_country.sort_values("Valeur", ascending=False).reset_index(drop=True)
    
    # Compared to EUR
    exchange_rate_by_country["Compared_with_EUR"] = exchange_rate_by_country["Valeur"]*0.92

    # Extract the top and bottom countries
    top_exchange_rate_by_country = exchange_rate_by_country.head(3)
    bottom_exchange_rate_by_country = exchange_rate_by_country.tail(3)
       
    # Print results
    print("Countries with the highest exchange rate for the year 2017:")
    for index, row in top_exchange_rate_by_country.iterrows():
        print(f"{row['Zone']}: {row['Compared_with_EUR']:.2f}")
        
    print("\nCountries with the lowest exchange rate for the year 2017:")
    for index, row in bottom_exchange_rate_by_country.iterrows():
        print(f"{row['Zone']}: {row['Compared_with_EUR']:.2f}")
        
    return exchange_rate_by_country

In [29]:
exchange_rate_result = exchange_rate_by_country(exchange_rate)

Countries with the highest exchange rate for the year 2017:
Iran (Islamic Republic of): 30568.19
Somalia: 21250.15
Viet Nam: 20580.48

Countries with the lowest exchange rate for the year 2017:
Oman: 0.35
Bahrain: 0.35
Kuwait: 0.28


In [30]:
def exchange_rate_evolution(exchange_rate):
    """
    Calculate the evolution of exchange rates.
    
    Args:
    - data (DataFrame): DataFrame containing exchange rate data.
        
    Returns:
        DataFrame: Evolution of exchange rates by country.
    """
    # Select only local currency units per USD
    exchange_rate = exchange_rate[exchange_rate["Unité"] == "Local currency units per USD"]
    
    # Convert columns to numeric data type
    exchange_rate["Valeur"] = pd.to_numeric(exchange_rate["Valeur"], errors="coerce")
    
    # Compared to EUR
    exchange_rate["Compared_with_EUR"] = exchange_rate["Valeur"]*0.92
    
    # Group by country and year and select the value column
    exchange_rate_grouped = exchange_rate.groupby(["Zone", "Année"])["Compared_with_EUR"].first().unstack()
    
    # Calculate percentage change over years
    exchange_rate_evol = exchange_rate_grouped.pct_change(axis='columns')
    
    # Calculate average evolution
    exchange_rate_evol["Change Average"] = exchange_rate_evol.mean(axis=1)

    # Reset index to avoid losing country names
    exchange_rate_evol = exchange_rate_evol.reset_index()
    
    # Remove infinite values
    exchange_rate_evol = exchange_rate_evol.replace(np.inf, np.nan)
    
    # Keep only the "Change Average" column
    exchange_rate_evol = exchange_rate_evol[["Zone", "Change Average"]]
    
    # Sort by average evolution
    exchange_rate_evol = exchange_rate_evol.sort_values("Change Average", ascending=False).reset_index(drop=True)
    
    # Extract the top and bottom countries
    top_exchange_rate_countries = exchange_rate_evol.head(3)
    bottom_exchange_rate_countries = exchange_rate_evol.tail(3)
    
    # Print results
    print("Countries with the highest exchange rate evolution :")
    for index, row in top_exchange_rate_countries.iterrows():
        print(f"{row['Zone']}: {row['Change Average']:.2f}")
        
    print("\nCountries with the lowest exchange rate evolution:")
    for index, row in bottom_exchange_rate_countries.iterrows():
        print(f"{row['Zone']}: {row['Change Average']:.2f}")
        
    return exchange_rate_evol

In [31]:
exchange_rate_evolution_result = exchange_rate_evolution(exchange_rate)

Countries with the highest exchange rate evolution :
Zimbabwe: 4094303802.32
Myanmar: 6.52
Democratic People's Republic of Korea: 3.80

Countries with the lowest exchange rate evolution:
Lithuania: -0.06
Greece: -0.07
Slovakia: -0.08


### 3.4 Consumer Price Index

In [32]:
def cpi_by_country(consumer_price_index):
    """
    Calculate the Consumer Price Index (CPI) by country.
    
    Args:
    - data (DataFrame): DataFrame containing CPI data.
        
    Returns:
        DataFrame: CPI by country.
    """
    # Filter the DataFrame for the year 2017
    cpi_2017 = consumer_price_index[(consumer_price_index["Année"] == "2017") & (consumer_price_index["Objet"] == "Food price inflation")]
    
    # Convert columns to numeric data type
    cpi_2017["Valeur"] = pd.to_numeric(cpi_2017["Valeur"], errors="coerce")
    
    # Group by country and mean the CPI values
    cpi_by_country = cpi_2017.groupby("Zone")["Valeur"].mean().reset_index()
    
    # Sort the DataFrame by CPI in descending order
    cpi_by_country = cpi_by_country.sort_values("Valeur", ascending=False).reset_index(drop=True)

    # Extract the top and bottom countries
    top_cpi_countries = cpi_by_country.head(3)
    bottom_cpi_countries = cpi_by_country.tail(3)
    
    # Print results
    print("Countries most impacted by inflation for the year 2017:")
    for index, row in top_cpi_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")
        
    print("\nCountries least impacted by inflation for the year 2017:")
    for index, row in bottom_cpi_countries.iterrows():
        print(f"{row['Zone']}: {row['Valeur']:.2f}")
       
    return cpi_by_country

In [33]:
cpi_country_result = cpi_by_country(consumer_price_index)

Countries most impacted by inflation for the year 2017:
Venezuela (Bolivarian Republic of): 480.86
South Sudan: 236.95
Egypt: 38.90

Countries least impacted by inflation for the year 2017:
Fiji: -2.08
Iceland: -2.41
Chad: -3.83


In [34]:
def cpi_evolution(consumer_price_index):
    """
    Calculate the evolution of the Consumer Price Index (CPI).
    
    Args:
    - data (DataFrame): DataFrame containing CPI data.
        
    Returns:
        DataFrame: Evolution of CPI by country.
    """
    # Filter only food price inflation
    consumer_prices = consumer_price_index[consumer_price_index["Objet"] == "Food price inflation"]
    
    # Convert the "Valeur" column to numeric
    consumer_prices["Valeur"] = pd.to_numeric(consumer_prices["Valeur"], errors="coerce")
    
    # Group by country and year and select the value column
    cpi_grouped = consumer_prices.groupby(["Zone", "Année"])["Valeur"].first().unstack()
    
    # Calculate percentage change over years
    cpi_evol = cpi_grouped.pct_change(axis='columns')
    
    # Calculate average evolution
    cpi_evol["CPI_Average_Evolution"] = cpi_evol.mean(axis=1)

    # Reset index
    cpi_evol = cpi_evol.reset_index()
    
    # Remove infinite and NaN values
    cpi_evol.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Keep only the "Change Average" column
    cpi_evol = cpi_evol[["Zone", "CPI_Average_Evolution"]]
    
    # Sort by average evolution
    cpi_evol = cpi_evol.sort_values("CPI_Average_Evolution", ascending=False).reset_index(drop=True)
    
    # Extract the top and bottom countries
    top_cpi_evol = cpi_evol.head(3)
    
    # Filter out NaN and inf values while selecting the bottom 3 countries
    bottom_cpi_evol = cpi_evol[~cpi_evol["CPI_Average_Evolution"].isna() & ~cpi_evol["CPI_Average_Evolution"].isin([np.inf, -np.inf])].tail(3)
    
    # Print results
    print("Countries with the highest inflation evolution:")
    for index, row in top_cpi_evol.iterrows():
        print(f"{row['Zone']}: {row['CPI_Average_Evolution']:.2f}")
        
    print("\nCountries with the lowest inflation evolution:")
    for index, row in bottom_cpi_evol.iterrows():
        print(f"{row['Zone']}: {row['CPI_Average_Evolution']:.2f}")
    
    return cpi_evol

In [35]:
cpi_evolution_result = cpi_evolution(consumer_price_index)

Countries with the highest inflation evolution:
Argentina: 83.88
Senegal: 49.94
Malta: 18.91

Countries with the lowest inflation evolution:
Nepal: -9.68
British Virgin Islands: -10.33
Madagascar: -13.61


### 3.5 Political Stability

In [36]:
def political_stability_by_country(political_stability):
    """
    Calculate political stability by country for the year 2017.
    
    Args:
    - data (DataFrame): DataFrame containing political stability data.
        
    Returns:
        DataFrame: Political stability by country.
    """
    # Filter the DataFrame for the year 2017
    political_stab_2017 = political_stability[political_stability["Année"] == "2017"]
    
    # Group by country and sum the political stability values
    political_stab_by_country = political_stab_2017.groupby("Zone")["Stabilité Politique"].sum().reset_index()
    
    # Convert the 'Stabilité Politique' column to numeric
    political_stab_by_country["Stabilité Politique"] = pd.to_numeric(political_stab_by_country["Stabilité Politique"], errors="coerce")
    
    # Sort the DataFrame by political stability in descending order
    political_stab_by_country_sorted = political_stab_by_country.sort_values("Stabilité Politique", ascending=False).reset_index(drop=True)
    
    # Extract the top and bottom countries
    top_stable_countries = political_stab_by_country_sorted.nlargest(3, "Stabilité Politique")
    bottom_stable_countries = political_stab_by_country_sorted.nsmallest(3, "Stabilité Politique")

    # Print results
    print("Countries with the best political stability:")
    for index, row in top_stable_countries.iterrows():
        print(f"{row['Zone']}: {row['Stabilité Politique']:.2f}")
        
    print("\nCountries with the worst political stability:")
    for index, row in bottom_stable_countries.iterrows():
        print(f"{row['Zone']}: {row['Stabilité Politique']:.2f}")
    
    return political_stab_by_country_sorted

In [37]:
political_stability_result = political_stability_by_country(political_stability)

Countries with the best political stability:
Greenland: 1.92
Singapore: 1.62
New Zealand: 1.59

Countries with the worst political stability:
Yemen: -2.94
Afghanistan: -2.80
Syrian Arab Republic: -2.62


### 3.6 Emissions

In [38]:
def food_pollution_evolution(emissions_totals):
    """
    Calculate the evolution of food-related pollution by country.
    
    Args:
    - data (DataFrame): DataFrame containing emissions data.
        
    Returns:
        DataFrame: Evolution of food-related pollution by country.
    """
    # Filter emissions for food transport and retail
    emissions = emissions_totals[(emissions_totals["Objet"] == "Food Transport") | (emissions_totals["Objet"] == "Food Retail")]
    
    # Convert 'Valeur' column to numeric
    emissions["Valeur"] = pd.to_numeric(emissions["Valeur"], errors="coerce")
    
    # Group by country and sum the emissions for each year
    emissions_grouped = emissions.groupby(["Zone", "Année"])["Valeur"].sum().unstack()
    
    # Calculate percentage change over years
    emissions_evol = emissions_grouped.pct_change(axis='columns')
    
    # Calculate average evolution
    emissions_evol["Average Evolution"] = emissions_evol.mean(axis=1)
    
    # Reset index to avoid losing country names
    emissions_evol = emissions_evol.reset_index()

    # Keep only the "Change Average" column
    emissions_evol = emissions_evol[["Zone", "Average Evolution"]]
    
    # Sort by average evolution
    emissions_evol_sorted = emissions_evol.sort_values("Average Evolution", ascending=False)

    # Extract the top and bottom countries
    top_polluting_countries = emissions_evol_sorted.head(3)
    bottom_polluting_countries = emissions_evol_sorted.tail(3)
    
    # Print results
    print("Countries with the highest pollution growth:")
    for index, row in top_polluting_countries.iterrows():
        print(f"{row['Zone']}: {row['Average Evolution']:.2f}")
        
    print("\nCountries with the lowest pollution growth:")
    for index, row in bottom_polluting_countries.iterrows():
        print(f"{row['Zone']}: {row['Average Evolution']:.2f}")
    
    return emissions_evol_sorted

In [39]:
food_pollution_result = food_pollution_evolution(emissions_totals)

Countries with the highest pollution growth:
Uganda: 0.21
Mauritius: 0.19
United Republic of Tanzania: 0.19

Countries with the lowest pollution growth:
Denmark: -0.01
Micronesia (Federated States of): -0.01
American Samoa: -0.02


In [40]:
def food_pollution_per_capita(emissions_totals, population_result):
    """
    Calculate food-related pollution per capita by country.
    
    Args:
    - data (DataFrame): DataFrame containing emissions data.
    - data (DataFrame): DataFrame containing population data for each country.
        
    Returns:
        DataFrame: Food-related pollution per capita by country.
    """
    # Filter emissions for the year 2017 and for food transport and retail
    emissions_totals = emissions_totals[(emissions_totals["Année"] == "2017") & 
                                        ((emissions_totals["Objet"] == "Food Transport") | 
                                         (emissions_totals["Objet"] == "Food Retail"))]
    
    # Convert 'Valeur' column to numeric
    emissions_totals["Valeur"] = pd.to_numeric(emissions_totals["Valeur"], errors="coerce")
    
    # Group emissions by country and sum the values
    emissions_by_country = emissions_totals.groupby("Zone").sum()["Valeur"].reset_index()
    
    # Merge emissions data with population data
    merged_data = pd.merge(emissions_by_country, population_result, on="Zone", how="right")
    merged_data["Population"] = pd.to_numeric(merged_data["Population"], errors="coerce")
    
    # Rename columns
    merged_data.rename(columns={"Valeur": "Emissions"}, inplace=True)
    
    # Calculate emissions per capita
    merged_data["Emissions_per_Capita"] = merged_data["Emissions"] / merged_data["Population"]
    
    # Drop rows where emissions is NaN
    merged_data.dropna(subset=["Emissions_per_Capita"], inplace=True)
    
    # Sort by emissions per capita
    merged_data = merged_data.sort_values("Emissions_per_Capita", ascending=False).reset_index(drop=True)
    
    # Print top and bottom countries with highest and lowest emissions per capita
    print("Countries with the highest emissions per capita:")
    for index, row in merged_data.head(3).iterrows():
        print(f"{row['Zone']}: {row['Emissions_per_Capita']:.2f}")
        
    print("\nCountries with the lowest emissions per capita:")
    for index, row in merged_data.tail(3).iterrows():
        print(f"{row['Zone']}: {row['Emissions_per_Capita']:.2f}")
    
    return merged_data

In [41]:
food_pollution_capita_result = food_pollution_per_capita(emissions_totals, population_result)

Countries with the highest emissions per capita:
Gibraltar: 3.80
Kuwait: 2.59
Qatar: 2.51

Countries with the lowest emissions per capita:
Eritrea: 0.02
Ethiopia: 0.01
Somalia: 0.00


## 4. Export<a class="anchor" id="partie4"></a>

### 4.1 Population

In [42]:
def export_population_data(population_result, population_growth_result):
    """
    Merge population data with growth information and export to DataFrame.
    
    Args:
    - population_result (DataFrame): DataFrame containing population data.
    - population_growth_result (DataFrame or Series): DataFrame or Series containing population growth information.
    
    Returns:
        DataFrame: Merged DataFrame with relevant columns.
    """
    # Reset index of Series inputs
    if isinstance(population_growth_result, pd.Series):
        population_growth_result = population_growth_result.reset_index(name='Change Average')

    # Merge population data with growth information
    merged_data = pd.merge(population_result, population_growth_result, how="left", on="Zone", suffixes=('_population', '_growth'))
    
    # Rename columns
    merged_data.rename(columns={"Valeur": "Population", "Change Average": "Population_Growth"}, inplace=True)
          
    # Find countries that were not merged
    missing_countries_result = set(population_growth_result["Zone"]) - set(population_result["Zone"])
    missing_countries_growth = set(population_result["Zone"]) - set(population_growth_result["Zone"])
    missing_countries = missing_countries_result.union(missing_countries_growth)
    
    # Print missing countries if they exist
    if missing_countries:
        print("Countries missed in the merge:", missing_countries)
    
    return merged_data

In [43]:
export_population_result = export_population_data(population_result, population_growth_result)

Countries missed in the merge: {'Serbia and Montenegro'}


### 4.2 Food Availability

In [44]:
def export_poultry_availability_data(poultry_availability_capita_result, poultry_imports_result, poultry_production_result, poultry_domestic_result):
    """
    Merge poultry availability data and export to DataFrame.
    
    Args:
    - poultry_availability_capita_result (DataFrame): DataFrame containing per capita poultry availability data.
    - poultry_imports_result (DataFrame): DataFrame containing poultry importation data by country.
    - poultry_production_result (DataFrame): DataFrame containing poultry production data by country.
    - poultry_domestic_result (DataFrame): DataFrame containing domestic poultry availability data by country.
    
    Returns:
        DataFrame: Merged DataFrame with relevant columns.
    """
    # Reset index of Series inputs
    if isinstance(poultry_availability_capita_result, pd.Series):
        poultry_availability_capita_result = poultry_availability_capita_result.reset_index(name='Disponibilité')
    if isinstance(poultry_imports_result, pd.Series):
        poultry_imports_result = poultry_imports_result.reset_index(name='Importations')
    if isinstance(poultry_production_result, pd.Series):
        poultry_production_result = poultry_production_result.reset_index(name='Production')
    if isinstance(poultry_domestic_result, pd.Series):
        poultry_domestic_result = poultry_domestic_result.reset_index(name='Domestic Availability')

    # Merge poultry_availability_capita_result with poultry_imports_result
    merged_data = pd.merge(poultry_availability_capita_result, poultry_imports_result, how="left", on="Zone", suffixes=('_availability', '_imports'))
    
    # Merge with poultry_production_result
    merged_data = pd.merge(merged_data, poultry_production_result, how="left", on="Zone", suffixes=('_imports', '_production'))
    
    # Merge with poultry_domestic_result
    merged_data = pd.merge(merged_data, poultry_domestic_result, how="left", on="Zone", suffixes=('_production', '_domestic'))
    
    # Rename columns
    merged_data.rename(columns={"Valeur_imports": "Importations",
                               "Valeur_production": "Production",
                               "Valeur": "Domestic Availability"}, inplace=True)
    
    # Find missing countries from each merge
    missing_countries_availability_imports = set(poultry_availability_capita_result["Zone"]) - set(poultry_imports_result["Zone"])
    missing_countries_imports_production = set(poultry_imports_result["Zone"]) - set(poultry_production_result["Zone"])
    missing_countries_production_domestic = set(poultry_production_result["Zone"]) - set(poultry_domestic_result["Zone"])
    
    # Print missing countries if they exist
    if missing_countries_availability_imports:
        print("Countries missed in the availability - imports merge:", missing_countries_availability_imports)
    if missing_countries_imports_production:
        print("Countries missed in the imports - production merge:", missing_countries_imports_production)
    if missing_countries_production_domestic:
        print("Countries missed in the production - domestic merge:", missing_countries_production_domestic)
    
    return merged_data[["Zone", "Poultry Availability", "Poultry_Availability_per_Capita", "Importations", "Production", "Domestic Availability"]]

In [45]:
export_poultry_result = export_poultry_availability_data(poultry_availability_capita_result, poultry_imports_result, poultry_production_result, poultry_domestic_result)

Countries missed in the availability - imports merge: {'Uzbekistan', "Lao People's Democratic Republic"}
Countries missed in the imports - production merge: {'Maldives', 'Djibouti'}


### 4.3 Exchange Rate

In [46]:
def export_exchange_rate_data(exchange_rate_evolution_result, exchange_rate_result):
    """
    Merge exchange rate data and export to DataFrame.
    
    Args:
    - exchange_rate_evolution_result (DataFrame): DataFrame containing exchange rate evolution data.
    - exchange_rate_result (DataFrame): DataFrame containing exchange rate data by country.
    
    Returns:
        DataFrame: Merged DataFrame with relevant columns.
    """
    # Reset index of Series inputs
    if isinstance(exchange_rate_evolution_result, pd.Series):
        exchange_rate_evolution_result = exchange_rate_evolution_result.reset_index(name='Valeur')
    if isinstance(exchange_rate_result, pd.Series):
         exchange_rate_result =  exchange_rate_result.reset_index(name='Change Average')
 
    # Merge exchange rate evolution data with exchange rate data by country
    merged_data = pd.merge(exchange_rate_evolution_result, exchange_rate_result, how="left", on="Zone", indicator=True)
    
    # Drop the "_merge" column if it exists
    merged_data = merged_data.drop("_merge", axis=1, errors="ignore")
    
    # Rename columns for consistency
    merged_data.rename(columns={"Valeur": "Exchange Rate", "Change Average": "Change Average"}, inplace=True)
    
    # Find missing countries
    missing_countries = set(exchange_rate_evolution_result["Zone"]) - set(exchange_rate_result["Zone"])
    
    # Print missing countries if they exist
    if missing_countries:
        print("Countries missed in the merge:", missing_countries)
    
    return merged_data

In [47]:
export_exchange_result = export_exchange_rate_data(exchange_rate_evolution_result, exchange_rate_result)

Countries missed in the merge: {'Netherlands Antilles', 'Venezuela (Bolivarian Republic of)', 'Sudan (former)'}


### 4.4 Consumer Price Index

In [48]:
def export_consumer_price_index_data(cpi_evolution_result, cpi_country_result):
    """
    Merge consumer price index data and export to DataFrame.
    
    Args:
    - cpi_evolution_result (DataFrame): DataFrame containing CPI evolution data.
    - cpi_by_country_result (DataFrame): DataFrame containing CPI data by country.
    
    Returns:
        DataFrame: Merged DataFrame with relevant columns.
    """
    # Reset index of Series inputs
    if isinstance(cpi_evolution_result, pd.Series):
        cpi_evolution_result = cpi_evolution_result.reset_index(name='Valeur')
    if isinstance(cpi_country_result, pd.Series):
         exchange_rate_result = cpi_country_result.reset_index(name='Change Average')
 
    # Merge CPI data
    merged_data = pd.merge(cpi_evolution_result, cpi_country_result, how="left", on="Zone", indicator=True)
    
    # Drop the "_merge" column if it exists
    merged_data = merged_data.drop("_merge", axis=1, errors="ignore")
    
    # Rename columns
    merged_data.rename(columns={"Valeur": "CPI", "Evol. Moyenne": "CPI Evolution"}, inplace=True)
    
    # Find missing countries
    missing_countries = set(cpi_evolution_result["Zone"]) - set(cpi_country_result["Zone"])
    
    # Print missing countries if they exist
    if missing_countries:
        print("Countries missed in the merge:", missing_countries)
        
    return merged_data

In [49]:
export_cpi_result = export_consumer_price_index_data(cpi_evolution_result, cpi_country_result)

### 4.5 Emissions

In [50]:
def export_emissions_data(country_coordinates, food_pollution_capita_result):
    """
    Merge emissions data and export to DataFrame.
    
    Args:
    - country_coordinates (DataFrame): DataFrame containing country coordinates data.
    - food_pollution_result (DataFrame): DataFrame containing food pollution data by country.
    
    Returns:
        DataFrame: Merged DataFrame with relevant columns.
    """
    # Keep only the "Km - Par rapport à la France" column in country_coordinates
    country_coordinates = country_coordinates[["Zone", "Km - Par rapport à la France"]]
    
    # Reset index of Series inputs
    if isinstance(food_pollution_capita_result, pd.Series):
        food_pollution_capita_result = food_pollution_capita_result.reset_index(name='Emissions')

    # Merge emissions data
    merged_data = pd.merge(country_coordinates, food_pollution_capita_result, how="right", on="Zone", indicator=True)
    
    # Drop the "_merge" column if it exists
    merged_data = merged_data.drop("_merge", axis=1, errors="ignore")
    
    # Find missing countries
    missing_countries = set(country_coordinates["Zone"]) - set(food_pollution_capita_result["Zone"])
    
    # Print missing countries if they exist
    if missing_countries:
        print("Countries missed in the merge:", missing_countries)
        
    return merged_data

In [51]:
export_emissions_result = export_emissions_data(country_coordinates, food_pollution_capita_result)

Countries missed in the merge: {'Burkina\xa0Faso', 'British Indian Ocean Territory', 'Pitcairn Islands', 'United States Minor Outlying Islands', 'Heard and McDonald Islands et Territory of Heard Island and McDonald Islands', 'French Southern and Antarctic Lands', 'Western Sahara', 'Cocos (Keeling) Islands', 'Kosovo', 'Guam', 'Tokelau', 'Guernesey', 'Antarctica', 'Svalbard and Jan Mayen', 'Bouvet Island', 'Saint-Marin', 'South Georgia and the South Sandwich Islands', 'Netherlands Antilles', 'Norfolk Island', 'Christmas Island', 'Jersey'}


## 5. Final Dataset

In [52]:
# Population + Emissions
population_emissions = pd.merge(export_population_result, export_emissions_result, how="left", on="Zone", indicator=True)

# Rename columns for clarity
population_emissions.rename(columns={"Km - Par rapport à la France": "Distance", "Average Evolution": "Emissions_per_Capita", "Population_x": "Population"}, inplace=True)

# Drop one of the Population columns
population_emissions.drop("Population_y", axis=1, inplace=True)

# Drop the "_merge" column if it exists
population_emissions.drop("_merge", axis=1, inplace=True)

# Find countries without emissions data
missing_emissions_data = population_emissions[population_emissions["Emissions"].isnull()]
num_missing_countries = missing_emissions_data.shape[0]
print(f"We have {num_missing_countries} countries without data on pollutant emissions.")
print(missing_emissions_data.head(3))
print('Population Emissions shape:', population_emissions.shape)

We have 11 countries without data on pollutant emissions.
                            Zone Population  Population_Growth  Distance  \
37         Caribbean Netherlands     25.401           0.022598       NaN   
41               Channel Islands    168.665           0.007729       NaN   
46  Collectivity of Saint Martin      36.56          -0.000616       NaN   

    Emissions  Emissions_per_Capita  
37        NaN                   NaN  
41        NaN                   NaN  
46        NaN                   NaN  
Population Emissions shape: (236, 6)


In [53]:
# Population + Emissions data with Exchange Rate data
population_emissions_change = pd.merge(population_emissions, export_exchange_result, how="left", on="Zone", indicator=True)

# Rename columns for clarity
population_emissions_change.rename(columns={"Change Average": "Exchange_Rate_Evolution"}, inplace=True)

# Drop the "_merge" column
population_emissions_change.drop("_merge", axis=1, inplace=True)

# Find countries without exchange rate data
missing_exchange_rate_data = population_emissions_change.loc[population_emissions_change["Exchange Rate"].isnull()]
num_missing_countries = missing_exchange_rate_data.shape[0]
print(f"We have {num_missing_countries} countries without exchange rate data.")
print(missing_exchange_rate_data.head(3))
print('Population Emissions Exchange Rate shape:', population_emissions_change.shape)

We have 17 countries without exchange rate data.
                     Zone Population  Population_Growth      Distance  \
3          American Samoa      55.62          -0.002280  16390.040913   
37  Caribbean Netherlands     25.401           0.022598           NaN   
41        Channel Islands    168.665           0.007729           NaN   

    Emissions  Emissions_per_Capita  Exchange_Rate_Evolution  Exchange Rate  \
3     27.1113              0.487438                      NaN            NaN   
37        NaN                   NaN                      NaN            NaN   
41        NaN                   NaN                      NaN            NaN   

    Compared_with_EUR  
3                 NaN  
37                NaN  
41                NaN  
Population Emissions Exchange Rate shape: (236, 9)


In [54]:
# Merge CPI data
pop_emissions_change_cpi = pd.merge(population_emissions_change, export_cpi_result, how="left", on="Zone", indicator=True)

# Rename columns for clarity
pop_emissions_change_cpi.rename(columns={"Evol. Moyenne": "CPI_Evolution"}, inplace=True)

# Drop the "_merge" column
pop_emissions_change_cpi.drop("_merge", axis=1, inplace=True)

# Find countries without CPI data
missing_cpi_data = pop_emissions_change_cpi.loc[pop_emissions_change_cpi["CPI"].isnull()]
num_missing_countries = missing_cpi_data.shape[0]
print(f"We have {num_missing_countries} countries without CPI data.")
print(missing_cpi_data.head(3))
print('Population Emissions Exchange Rate CPI shape:', pop_emissions_change_cpi.shape)

We have 35 countries without CPI data.
                        Zone Population  Population_Growth      Distance  \
3             American Samoa      55.62          -0.002280  16390.040913   
37     Caribbean Netherlands     25.401           0.022598           NaN   
39  Central African Republic   4596.023           0.013911   4744.553501   

    Emissions  Emissions_per_Capita  Exchange_Rate_Evolution  Exchange Rate  \
3     27.1113              0.487438                      NaN            NaN   
37        NaN                   NaN                      NaN            NaN   
39   466.1214              0.101418                -0.010665      580.65675   

    Compared_with_EUR  CPI_Average_Evolution  CPI  
3                 NaN                    NaN  NaN  
37                NaN                    NaN  NaN  
39          534.20421                    NaN  NaN  
Population Emissions Exchange Rate CPI shape: (236, 11)


In [55]:
# Merge Political Stability data
pop_em_change_cpi_pol_stab = pd.merge(pop_emissions_change_cpi, political_stability_result, how="left", on="Zone", indicator=True)

# Rename columns for clarity
pop_em_change_cpi_pol_stab.rename(columns={"Stabilité Politique": "Political Stability"}, inplace=True)

# Drop the "_merge" column
pop_em_change_cpi_pol_stab.drop("_merge", axis=1, inplace=True)

# Find countries without political stability data
missing_stability_data = pop_em_change_cpi_pol_stab.loc[pop_em_change_cpi_pol_stab["Political Stability"].isnull()]
num_missing_countries = missing_stability_data.shape[0]
print(f"We have {num_missing_countries} countries without political stability data.")
print(missing_stability_data.head(3))
print('Population Emissions Exchange Rate CPI Political Stability shape:', pop_em_change_cpi_pol_stab.shape)

We have 38 countries without political stability data.
                      Zone Population  Population_Growth     Distance  \
6                 Anguilla     14.584           0.015087  6674.510099   
10                   Aruba    105.366           0.008547  7644.973573   
28  British Virgin Islands     29.577           0.021615  6780.265285   

    Emissions  Emissions_per_Capita  Exchange_Rate_Evolution  Exchange Rate  \
6     20.5417              1.408509                      0.0           2.70   
10   233.0529              2.211842                      0.0           1.79   
28    22.9247              0.775085                      0.0           1.00   

    Compared_with_EUR  CPI_Average_Evolution       CPI  Political Stability  
6              2.4840              -0.241167  1.203278                  NaN  
10             1.6468              -1.294417 -1.999478                  NaN  
28             0.9200             -10.334447 -1.825296                  NaN  
Population Emissions Ex

In [56]:
# Merge Availability Data
merged_df = pd.merge(pop_em_change_cpi_pol_stab, export_poultry_result, how="left", on="Zone", indicator=True)

# Drop the "_merge" column
merged_df.drop("_merge", axis=1, inplace=True)

# Print countries without availability data
missing_availability = merged_df.loc[merged_df["Poultry Availability"].isnull()]
print(f"We have {missing_availability.shape[0]} countries without food availability data.")
print(missing_availability.sort_values("Population", ascending=False).head(3))
print('Merged dataframe shape:', merged_df.shape)

We have 64 countries without food availability data.
                 Zone Population  Population_Growth     Distance  Emissions  \
188        Seychelles     96.418           0.010137  7696.434420    72.2679   
176  Saint Barthélemy      9.784           0.007084          NaN        NaN   
175           Réunion    876.134           0.010088  9186.545694   323.0959   

     Emissions_per_Capita  Exchange_Rate_Evolution  Exchange Rate  \
188              0.749527                 0.059889      13.647842   
176                   NaN                      NaN            NaN   
175              0.368775                -0.010665       0.885206   

     Compared_with_EUR  CPI_Average_Evolution       CPI  Political Stability  \
188          12.556014               4.787759  0.349697                 0.79   
176                NaN                    NaN       NaN                  NaN   
175           0.814389                    NaN  0.920770                  NaN   

     Poultry Availability  Poult

In [57]:
df = merged_df[merged_df['Poultry Availability'].notna()]
print("Number of countries after removing missing availability data:", df.shape[0])

Number of countries after removing missing availability data: 172


In [58]:
# Convert 'Population' to float
print('Format of the dataframe before conversion:')
print(df.info())

df['Population'] = pd.to_numeric(df['Population'], errors='coerce')

# Print the information about the dataframe after conversion
print('\nFormat of the dataframe after conversion:')
print(df.info())

Format of the dataframe before conversion:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 235
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Zone                             172 non-null    object 
 1   Population                       172 non-null    object 
 2   Population_Growth                172 non-null    float64
 3   Distance                         171 non-null    float64
 4   Emissions                        172 non-null    float64
 5   Emissions_per_Capita             172 non-null    float64
 6   Exchange_Rate_Evolution          172 non-null    float64
 7   Exchange Rate                    171 non-null    float64
 8   Compared_with_EUR                171 non-null    float64
 9   CPI_Average_Evolution            161 non-null    float64
 10  CPI                              166 non-null    float64
 11  Political Stability              170 non-

In [59]:
def treat_outliers(df):
    # Check for non-numeric columns
    non_numeric_columns = df.select_dtypes(exclude=['float64', 'int64']).columns.tolist()
    if non_numeric_columns:
        print(f"Non-numeric columns detected: {', '.join(non_numeric_columns)}. They will be excluded from outlier treatment.")
    
    # Treat outliers only for 'Exchange_Rate_Evolution' and 'CPI' columns
    for column in ['Exchange_Rate_Evolution', 'CPI']:
        if column in df.columns:
            q1 = df[column].quantile(0.25)
            q3 = df[column].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
            if not outliers.empty:
                median_value = df[column].median()
                df.loc[outliers.index, column] = median_value

In [60]:
# Assuming df is your DataFrame
treat_outliers(df)

Non-numeric columns detected: Zone. They will be excluded from outlier treatment.


In [61]:
def treat_missing_values(df):
    # Calculate the percentage of missing values in the dataset
    missing_values_percentage = df.isna().mean()
    print('Percentage of missing values in the dataset:')
    print(missing_values_percentage)
    
    # Treat missing values in CPI_Average_Evolution and CPI columns
    mean_cpi_evolution = df['CPI_Average_Evolution'].mean()
    mean_cpi = df['CPI'].mean()
    df['CPI_Average_Evolution'].fillna(mean_cpi_evolution, inplace=True)
    df['CPI'].fillna(mean_cpi, inplace=True)
    
    # Replace remaining missing values with 0
    df.fillna(0, inplace=True)
    
    # Check the percentage of missing values again after replacing them with 0
    print('\nPercentage of missing values after replacing with 0:')
    print(df.isna().mean())

In [62]:
treat_missing_values(df)

Percentage of missing values in the dataset:
Zone                               0.000000
Population                         0.000000
Population_Growth                  0.000000
Distance                           0.005814
Emissions                          0.000000
Emissions_per_Capita               0.000000
Exchange_Rate_Evolution            0.000000
Exchange Rate                      0.005814
Compared_with_EUR                  0.005814
CPI_Average_Evolution              0.063953
CPI                                0.034884
Political Stability                0.011628
Poultry Availability               0.000000
Poultry_Availability_per_Capita    0.000000
Importations                       0.011628
Production                         0.023256
Domestic Availability              0.011628
dtype: float64

Percentage of missing values after replacing with 0:
Zone                               0.0
Population                         0.0
Population_Growth                  0.0
Distance             

In [63]:
df = df[["Zone", "Population", "Population_Growth", "Emissions", "Emissions_per_Capita", "Distance", "Compared_with_EUR", "Exchange_Rate_Evolution", "CPI", "CPI_Average_Evolution", "Political Stability", "Poultry Availability", "Poultry_Availability_per_Capita", "Importations", "Production", "Domestic Availability"]]
df.to_csv("df_for_cluster.csv", index=False)

,Zone,Population,Population_Growth,Emissions,Emissions_per_Capita,Distance,Compared_with_EUR,Exchange_Rate_Evolution,CPI,CPI_Average_Evolution,Political Stability,Poultry Availability,Poultry_Availability_per_Capita,Importations,Production,Domestic Availability
0,Afghanistan,36296.113,0.032864,1646.2889,0.045357,5585.579825,62.584752,-0.032702,6.957529,0.132833,-2.80,5.0,0.000138,29.0,28.0,57.0
1,Albania,2884.169,-0.004542,906.7435,0.314386,1549.151949,109.572000,-0.012120,3.924577,-4.817490,0.38,85.0,0.029471,38.0,13.0,47.0
2,Algeria,41389.189,0.017248,11062.8003,0.267287,2019.778162,102.095175,0.027047,5.040496,-0.524376,-0.92,22.0,0.000532,2.0,275.0,277.0
